In [ ]:
from typing import List, Any, TypeVar, Callable
import time
import random
import matplotlib.pyplot as plt
import numpy as np

T = TypeVar('T')

def measure_time(sort_func: Callable[[List[T]], List[T]], arr: List[T]) -> float:
    """Measure the execution time of a sorting function."""
    start_time = time.time()
    sort_func(arr.copy())  # Use a copy to avoid modifying the original array
    end_time = time.time()
    return end_time - start_time

def is_sorted(arr: List[T]) -> bool:
    """Check if an array is sorted in ascending order."""
    return all(arr[i] <= arr[i+1] for i in range(len(arr)-1))

def generate_random_array(size: int, min_val: int = 0, max_val: int = 1000) -> List[int]:
    """Generate a random array of integers."""
    return [random.randint(min_val, max_val) for _ in range(size)]

def generate_nearly_sorted_array(size: int, swaps: int) -> List[int]:
    """Generate a nearly sorted array by making a few swaps in a sorted array."""
    arr = list(range(size))
    for _ in range(swaps):
        i, j = random.sample(range(size), 2)
        arr[i], arr[j] = arr[j], arr[i]
    return arr

def generate_reversed_array(size: int) -> List[int]:
    """Generate a reversed sorted array."""
    return list(range(size, 0, -1))

def compare_sorting_algorithms(algorithms: List[Callable], sizes: List[int], 
                              data_type: str = 'random') -> None:
    """Compare the performance of different sorting algorithms on arrays of varying sizes."""
    times = {func.__name__: [] for func in algorithms}
    
    for size in sizes:
        print(f"Testing with array size: {size}")
        
        # Generate test data based on type
        if data_type == 'random':
            arr = generate_random_array(size)
        elif data_type == 'nearly_sorted':
            arr = generate_nearly_sorted_array(size, size // 10)
        elif data_type == 'reversed':
            arr = generate_reversed_array(size)
        else:
            raise ValueError("Invalid data type")
        
        for func in algorithms:
            # Skip slow algorithms for large arrays
            if size > 10000 and func.__name__ in ['bubble_sort', 'selection_sort', 'insertion_sort']:
                times[func.__name__].append(None)
                continue
                
            time_taken = measure_time(func, arr)
            times[func.__name__].append(time_taken)
            print(f"  {func.__name__}: {time_taken:.6f} seconds")
    
    # Plot results
    plt.figure(figsize=(12, 8))
    
    for func in algorithms:
        # Filter out None values
        valid_times = [(s, t) for s, t in zip(sizes, times[func.__name__]) if t is not None]
        if valid_times:
            valid_sizes, valid_time_values = zip(*valid_times)
            plt.plot(valid_sizes, valid_time_values, marker='o', label=func.__name__)
    
    plt.title(f'Sorting Algorithm Performance ({data_type} data)')
    plt.xlabel('Array Size')
    plt.ylabel('Time (seconds)')
    plt.legend()
    plt.grid(True)
    plt.xscale('log')
    plt.yscale('log')
    
    # Add annotations for complexity classes
    x = np.logspace(np.log10(min(sizes)), np.log10(max(sizes)), 100)
    
    # O(n^2)
    y_n2 = [x_i**2 * 1e-7 for x_i in x]
    plt.plot(x, y_n2, 'k--', alpha=0.3, label='O(n²)')
    
    # O(n log n)
    y_nlogn = [x_i * np.log2(x_i) * 1e-6 for x_i in x]
    plt.plot(x, y_nlogn, 'k:', alpha=0.3, label='O(n log n)')
    
    plt.legend()
    plt.tight_layout()
    plt.show()


In [ ]:
from typing import List, Any, TypeVar

T = TypeVar('T')

def bubble_sort(arr: List[T]) -> List[T]:
    """
    Sort an array using the bubble sort algorithm.
    
    Args:
        arr: List to be sorted
        
    Returns:
        Sorted list
    """
    n = len(arr)
    arr = arr.copy()  # Create a copy to avoid modifying the original
    
    for i in range(n):
        # Flag to optimize if array is already sorted
        swapped = False
        
        # Last i elements are already in place
        for j in range(0, n - i - 1):
            if arr[j] > arr[j + 1]:
                arr[j], arr[j + 1] = arr[j + 1], arr[j]
                swapped = True
        
        # If no swapping occurred in this pass, array is sorted
        if not swapped:
            break
    
    return arr

def selection_sort(arr: List[T]) -> List[T]:
    """
    Sort an array using the selection sort algorithm.
    
    Args:
        arr: List to be sorted
        
    Returns:
        Sorted list
    """
    n = len(arr)
    arr = arr.copy()  # Create a copy to avoid modifying the original
    
    for i in range(n):
        # Find the minimum element in the unsorted part
        min_idx = i
        for j in range(i + 1, n):
            if arr[j] < arr[min_idx]:
                min_idx = j
        
        # Swap the found minimum element with the first element
        arr[i], arr[min_idx] = arr[min_idx], arr[i]
    
    return arr

def insertion_sort(arr: List[T]) -> List[T]:
    """
    Sort an array using the insertion sort algorithm.
    
    Args:
        arr: List to be sorted
        
    Returns:
        Sorted list
    """
    arr = arr.copy()  # Create a copy to avoid modifying the original
    
    for i in range(1, len(arr)):
        key = arr[i]
        j = i - 1
        
        # Move elements greater than key one position ahead
        while j >= 0 and arr[j] > key:
            arr[j + 1] = arr[j]
            j -= 1
        
        arr[j + 1] = key
    
    return arr

# Demonstrate simple sorting algorithms
def demo_simple_sorts():
    print("Simple Sorting Algorithms Demonstration")
    print("====================================")
    
    # Test arrays
    arrays = [
        [5, 2, 9, 1, 5, 6],
        [1, 2, 3, 4, 5],
        [5, 4, 3, 2, 1],
        [3, 3, 3, 3, 3],
        []
    ]
    
    array_names = ["Random array", "Already sorted", "Reverse sorted", "All equal elements", "Empty array"]
    
    for arr, name in zip(arrays, array_names):
        print(f"\n{name}: {arr}")
        
        # Apply each sorting algorithm
        bubble_result = bubble_sort(arr)
        selection_result = selection_sort(arr)
        insertion_result = insertion_sort(arr)
        
        print(f"Bubble sort:    {bubble_result}")
        print(f"Selection sort: {selection_result}")
        print(f"Insertion sort: {insertion_result}")
        
        # Verify results
        expected = sorted(arr)
        assert bubble_result == expected, "Bubble sort failed"
        assert selection_result == expected, "Selection sort failed"
        assert insertion_result == expected, "Insertion sort failed"
    
    # Compare performance on larger arrays
    print("\nPerformance comparison on larger arrays:")
    sizes = [100, 1000]
    
    for size in sizes:
        print(f"\nArray size: {size}")
        
        # Generate a random array
        random_array = generate_random_array(size)
        
        # Measure execution time for each algorithm
        bubble_time = measure_time(bubble_sort, random_array)
        selection_time = measure_time(selection_sort, random_array)
        insertion_time = measure_time(insertion_sort, random_array)
        
        print(f"Bubble sort:    {bubble_time:.6f} seconds")
        print(f"Selection sort: {selection_time:.6f} seconds")
        print(f"Insertion sort: {insertion_time:.6f} seconds")
    
    # Test adaptivity with nearly sorted array
    print("\nAdaptivity test with nearly sorted array:")
    nearly_sorted = generate_nearly_sorted_array(1000, 50)
    
    bubble_time = measure_time(bubble_sort, nearly_sorted)
    selection_time = measure_time(selection_sort, nearly_sorted)
    insertion_time = measure_time(insertion_sort, nearly_sorted)
    
    print(f"Bubble sort:    {bubble_time:.6f} seconds")
    print(f"Selection sort: {selection_time:.6f} seconds")
    print(f"Insertion sort: {insertion_time:.6f} seconds")

# Run the demonstration
demo_simple_sorts()


In [ ]:
from typing import List, TypeVar, Optional

T = TypeVar('T')

def merge_sort(arr: List[T]) -> List[T]:
    """
    Sort an array using the merge sort algorithm.
    
    Args:
        arr: List to be sorted
        
    Returns:
        Sorted list
    """
    arr = arr.copy()  # Create a copy to avoid modifying the original
    
    if len(arr) <= 1:
        return arr
    
    # Divide array into two halves
    mid = len(arr) // 2
    left = merge_sort(arr[:mid])
    right = merge_sort(arr[mid:])
    
    # Merge the sorted halves
    return merge(left, right)

def merge(left: List[T], right: List[T]) -> List[T]:
    """
    Merge two sorted arrays into one sorted array.
    
    Args:
        left: First sorted array
        right: Second sorted array
        
    Returns:
        Merged sorted array
    """
    result = []
    i = j = 0
    
    while i < len(left) and j < len(right):
        if left[i] <= right[j]:
            result.append(left[i])
            i += 1
        else:
            result.append(right[j])
            j += 1
    
    # Add remaining elements
    result.extend(left[i:])
    result.extend(right[j:])
    
    return result

def quick_sort(arr: List[T]) -> List[T]:
    """
    Sort an array using the quick sort algorithm.
    
    Args:
        arr: List to be sorted
        
    Returns:
        Sorted list
    """
    arr = arr.copy()  # Create a copy to avoid modifying the original
    
    if len(arr) <= 1:
        return arr
    
    pivot = arr[len(arr) // 2]
    left = [x for x in arr if x < pivot]
    middle = [x for x in arr if x == pivot]
    right = [x for x in arr if x > pivot]
    
    return quick_sort(left) + middle + quick_sort(right)

def quick_sort_in_place(arr: List[T], low: int = 0, high: Optional[int] = None) -> List[T]:
    """
    Sort an array in-place using the quick sort algorithm.
    
    Args:
        arr: List to be sorted
        low: Starting index
        high: Ending index
        
    Returns:
        Sorted list
    """
    arr = arr.copy()  # Create a copy to avoid modifying the original
    
    if high is None:
        high = len(arr) - 1
    
    def _quick_sort(arr, low, high):
        if low < high:
            pivot_idx = _partition(arr, low, high)
            _quick_sort(arr, low, pivot_idx - 1)
            _quick_sort(arr, pivot_idx + 1, high)
    
    def _partition(arr, low, high):
        pivot = arr[high]
        i = low - 1
        
        for j in range(low, high):
            if arr[j] <= pivot:
                i += 1
                arr[i], arr[j] = arr[j], arr[i]
        
        arr[i + 1], arr[high] = arr[high], arr[i + 1]
        return i + 1
    
    _quick_sort(arr, low, high)
    return arr

def heap_sort(arr: List[T]) -> List[T]:
    """
    Sort an array using the heap sort algorithm.
    
    Args:
        arr: List to be sorted
        
    Returns:
        Sorted list
    """
    arr = arr.copy()  # Create a copy to avoid modifying the original
    n = len(arr)
    
    # Build max heap
    for i in range(n // 2 - 1, -1, -1):
        _heapify(arr, n, i)
    
    # Extract elements one by one
    for i in range(n - 1, 0, -1):
        arr[0], arr[i] = arr[i], arr[0]  # Swap
        _heapify(arr, i, 0)
    
    return arr

def _heapify(arr: List[T], n: int, i: int) -> None:
    """
    Heapify subtree rooted at index i.
    
    Args:
        arr: Array representation of heap
        n: Size of heap
        i: Root index
    """
    largest = i
    left = 2 * i + 1
    right = 2 * i + 2
    
    # Check if left child exists and is greater than root
    if left < n and arr[left] > arr[largest]:
        largest = left
    
    # Check if right child exists and is greater than largest so far
    if right < n and arr[right] > arr[largest]:
        largest = right
    
    # Change root if needed
    if largest != i:
        arr[i], arr[largest] = arr[largest], arr[i]
        _heapify(arr, n, largest)

# Demonstrate efficient sorting algorithms
def demo_efficient_sorts():
    print("Efficient Sorting Algorithms Demonstration")
    print("=====================================")
    
    # Test arrays
    arrays = [
        [5, 2, 9, 1, 5, 6],
        [1, 2, 3, 4, 5],
        [5, 4, 3, 2, 1],
        [3, 3, 3, 3, 3],
        []
    ]
    
    array_names = ["Random array", "Already sorted", "Reverse sorted", "All equal elements", "Empty array"]
    
    for arr, name in zip(arrays, array_names):
        print(f"\n{name}: {arr}")
        
        # Apply each sorting algorithm
        merge_result = merge_sort(arr)
        quick_result = quick_sort(arr)
        quick_in_place_result = quick_sort_in_place(arr)
        heap_result = heap_sort(arr)
        
        print(f"Merge sort:           {merge_result}")
        print(f"Quick sort:           {quick_result}")
        print(f"Quick sort (in-place): {quick_in_place_result}")
        print(f"Heap sort:            {heap_result}")
        
        # Verify results
        expected = sorted(arr)
        assert merge_result == expected, "Merge sort failed"
        assert quick_result == expected, "Quick sort failed"
        assert quick_in_place_result == expected, "Quick sort (in-place) failed"
        assert heap_result == expected, "Heap sort failed"
    
    # Compare performance on larger arrays
    print("\nPerformance comparison on larger arrays:")
    sizes = [1000, 10000]
    
    for size in sizes:
        print(f"\nArray size: {size}")
        
        # Generate a random array
        random_array = generate_random_array(size)
        
        # Measure execution time for each algorithm
        merge_time = measure_time(merge_sort, random_array)
        quick_time = measure_time(quick_sort, random_array)
        quick_in_place_time = measure_time(quick_sort_in_place, random_array)
        heap_time = measure_time(heap_sort, random_array)
        
        print(f"Merge sort:            {merge_time:.6f} seconds")
        print(f"Quick sort:            {quick_time:.6f} seconds")
        print(f"Quick sort (in-place): {quick_in_place_time:.6f} seconds")
        print(f"Heap sort:             {heap_time:.6f} seconds")
    
    # Test with different input patterns
    print("\nPerformance with different input patterns (size=10000):")
    
    # Nearly sorted array
    print("\nNearly sorted array:")
    nearly_sorted = generate_nearly_sorted_array(10000, 100)
    
    merge_time = measure_time(merge_sort, nearly_sorted)
    quick_time = measure_time(quick_sort, nearly_sorted)
    quick_in_place_time = measure_time(quick_sort_in_place, nearly_sorted)
    heap_time = measure_time(heap_sort, nearly_sorted)
    
    print(f"Merge sort:            {merge_time:.6f} seconds")
    print(f"Quick sort:            {quick_time:.6f} seconds")
    print(f"Quick sort (in-place): {quick_in_place_time:.6f} seconds")
    print(f"Heap sort:             {heap_time:.6f} seconds")
    
    # Reversed array
    print("\nReversed array:")
    reversed_array = generate_reversed_array(10000)
    
    merge_time = measure_time(merge_sort, reversed_array)
    quick_time = measure_time(quick_sort, reversed_array)
    quick_in_place_time = measure_time(quick_sort_in_place, reversed_array)
    heap_time = measure_time(heap_sort, reversed_array)
    
    print(f"Merge sort:            {merge_time:.6f} seconds")
    print(f"Quick sort:            {quick_time:.6f} seconds")
    print(f"Quick sort (in-place): {quick_in_place_time:.6f} seconds")
    print(f"Heap sort:             {heap_time:.6f} seconds")

# Run the demonstration
demo_efficient_sorts()
